In [1]:
import re
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return ' '.join([i.strip() for i in filter(None, text.split())])

clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))
    
min_line_length = 2
max_line_length = 5
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

question_test = short_questions[500:550]
answer_test = short_answers[500:550]
short_questions = short_questions[:500]
short_answers = short_answers[:500]

'what good stuff'

In [27]:
combined = []
for i in range(len(short_questions)):
    combined.append('%s %s <END>'%(short_questions[i],short_answers[i]))

In [28]:
from collections import *

def train_chatbot(data, order=4):
    lm = defaultdict(Counter)
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[' '.join(history)][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [29]:
%%time
lm = train_chatbot((' '.join(combined)).split(), order=1)

CPU times: user 9.4 ms, sys: 39 µs, total: 9.44 ms
Wall time: 8.88 ms


In [32]:
lm['you']

[('read', 0.00975609756097561),
 ('take', 0.00975609756097561),
 ('think', 0.02926829268292683),
 ('a', 0.004878048780487805),
 ('anytime', 0.004878048780487805),
 ('know', 0.01951219512195122),
 ('considered', 0.004878048780487805),
 ('i', 0.024390243902439025),
 ('lie', 0.004878048780487805),
 ('guys', 0.004878048780487805),
 ('do', 0.02926829268292683),
 ('religious', 0.004878048780487805),
 ('noticed', 0.00975609756097561),
 ('feeling', 0.004878048780487805),
 ('been', 0.004878048780487805),
 ('okay', 0.00975609756097561),
 ('go', 0.014634146341463415),
 ('want', 0.024390243902439025),
 ('said', 0.00975609756097561),
 ('will', 0.004878048780487805),
 ('cannot', 0.00975609756097561),
 ('oh', 0.004878048780487805),
 ('give', 0.00975609756097561),
 ('and', 0.004878048780487805),
 ('have', 0.04878048780487805),
 ('yes', 0.004878048780487805),
 ('bring', 0.004878048780487805),
 ('hit', 0.004878048780487805),
 ('herr', 0.004878048780487805),
 ('dream', 0.004878048780487805),
 ('played', 

In [33]:
from random import random

def generate_word(lm, history, order):
    history = history[-order:]
    dist = lm[history]
    x = random()
    for c,v in dist:
        x = x - v
        if x <= 0: return c 

In [34]:
dist = lm['you']

In [35]:
for c,v in dist:
    print(c,v)

read 0.00975609756097561
take 0.00975609756097561
think 0.02926829268292683
a 0.004878048780487805
anytime 0.004878048780487805
know 0.01951219512195122
considered 0.004878048780487805
i 0.024390243902439025
lie 0.004878048780487805
guys 0.004878048780487805
do 0.02926829268292683
religious 0.004878048780487805
noticed 0.00975609756097561
feeling 0.004878048780487805
been 0.004878048780487805
okay 0.00975609756097561
go 0.014634146341463415
want 0.024390243902439025
said 0.00975609756097561
will 0.004878048780487805
cannot 0.00975609756097561
oh 0.004878048780487805
give 0.00975609756097561
and 0.004878048780487805
have 0.04878048780487805
yes 0.004878048780487805
bring 0.004878048780487805
hit 0.004878048780487805
herr 0.004878048780487805
dream 0.004878048780487805
played 0.00975609756097561
defend 0.004878048780487805
calling 0.004878048780487805
look 0.004878048780487805
promise 0.014634146341463415
set 0.00975609756097561
stay 0.004878048780487805
identified 0.004878048780487805
h

In [ ]:
def generate_reply(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)